In [1]:
lib = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/'
import sys
sys.path.append(lib)
from utilities.package_importing import *
import scipy.stats as stats
import xgboost as xgb
from MachineLearning.ML_utilities.dataloder import *
from MachineLearning.ML_utilities.general_utilities import *
from MachineLearning.Models.enhanced_xgboost import *

In [10]:
EXP_FOLDER = r'/storage/md_keren/shitay/outputs/ML/DEBUG'

COHORT_PATH = r'/storage/md_keren/shitay/Data/droplet_seq/M97_M173/cohort/normalized/16.3.22/immune_cells_normalized_16.3.22_4k_genes.pkl'
SPLIT_DATA_PATH = r'/storage/md_keren/shitay/Data/tables/ML/split_into_test_val_train_sets/cluster_mapping'
CLUSTER_NUM = 13 # cluster 13 - all clusters combined
MARKERS_FOLDER_PATH = r'/storage/md_keren/shitay/outputs/response_analysis/integrated_cohort_4.11.21'


### XGBOOST:
NUM_ROUND =  40
MAX_DEPTH = 5
EARLY_STOPPING_ROUNDS =  5


In [11]:
def use_markers_to_filter_cohort(cohort, MARKERS_FOLDER_PATH):
    NON_RESPONSE_MARKER_PATH = join(MARKERS_FOLDER_PATH, r'non_response_immune_markers.xlsx')
    RESPONSE_MARKER_PATH = join(MARKERS_FOLDER_PATH, r'response_immune_markers.xlsx')
    non_response_markers = pd.read_excel(NON_RESPONSE_MARKER_PATH)
    response_markers = pd.read_excel(RESPONSE_MARKER_PATH)
    response_markers_indexes = [cohort.features.index(feature) for feature in response_markers.features if feature in cohort.features]
    non_response_markers_indexes = [cohort.features.index(feature) for feature in non_response_markers.features if feature in cohort.features]
    marker_indexes = response_markers_indexes + non_response_markers_indexes[:66]
    cohort.filter_gene_by_indexes(marker_indexes)

def build_datasets(cohort):
    # extracts params.
    if MARKERS_FOLDER_PATH:
        use_markers_to_filter_cohort(cohort, MARKERS_FOLDER_PATH)
    data_loader = Droplet_data_manager(cohort, SPLIT_DATA_PATH, CLUSTER_NUM)
    return data_loader


In [12]:
cohort = pickle.load(open(COHORT_PATH, 'rb'))

In [13]:
data_loader = build_datasets(cohort)
x_train, x_test, y_train, y_test = data_loader.train_X, data_loader.test_X, data_loader.train_Y, data_loader.test_Y
x_val, y_val = data_loader.validation_X, data_loader.validation_Y
feature_names = data_loader.feature_names



Dataset was cleared, left with 100 genes with variance of less than
Using clinical table in path:
 /storage/md_keren/shitay/Data/tables/clinical_tables/Melanoma_clinical_data_3.22_unprotected.xlsx

and labels:
/storage/md_keren/shitay/Data/tables/clinical_tables/clinical_labels.csv


In [14]:
model = NGModel(feature_names, NUM_ROUND, EARLY_STOPPING_ROUNDS, MAX_DEPTH)

In [15]:
model.train(x_train, y_train, x_val, y_val, verbose=False)


In [16]:
# xgb.plot_tree(model.model, num_trees=2)
# plt.show()

In [ ]:
## cluster 13 (all) with gene marker filter

In [17]:
# Inferences on train set.

print("Train inference")
y_pred = model.inference(x_train)
Metrics(y_train, y_pred).print_scores('Train cell')

print("val inference")
y_pred = model.inference(x_val)
Metrics(y_val, y_pred).print_scores('val cell')

print("Test inference")
y_pred = model.inference(x_test)
Metrics(y_test, y_pred).print_scores('Test cell')

Train inference
------------------------
Train cell scores:
Accuracy: 0.8176003101376236
sensitivity: 0.7773190869354055
specificity: 0.8443548387096774
precision: 0.7683629380700913
Confusion matrix:
[[5235  965]
 [ 917 3201]]
------------------------
val inference
------------------------
val cell scores:
Accuracy: 0.6559821095788296
sensitivity: 0.5383795309168443
specificity: 0.7191977077363897
precision: 0.507537688442211
Confusion matrix:
[[1255  490]
 [ 433  505]]
------------------------
Test inference
------------------------
Test cell scores:
Accuracy: 0.4732834536184864
sensitivity: 0.4794922322533339
specificity: 0.4571258199165176
precision: 0.6968364968364968
Confusion matrix:
[[ 3833  4552]
 [11358 10463]]
------------------------


In [9]:
# cluster 5 no gene marker filter

Train inference
------------------------
Train cell scores:
Accuracy: 0.9910066724688135
sensitivity: 1.0
specificity: 0.9860548807917229
precision: 0.9752988047808765
Confusion matrix:
[[2192   31]
 [   0 1224]]
------------------------
val inference
------------------------
val cell scores:
Accuracy: 0.6944444444444444
sensitivity: 0.5737704918032787
specificity: 0.7771535580524345
precision: 0.6382978723404256
Confusion matrix:
[[415 119]
 [156 210]]
------------------------
Test inference
------------------------
Test cell scores:
Accuracy: 0.4726166328600406
sensitivity: 0.3779385171790235
specificity: 0.6416464891041163
precision: 0.653125
Confusion matrix:
[[ 795  444]
 [1376  836]]
------------------------


In [21]:
# cluster 5 with gene marker filter

Train inference
------------------------
Train cell scores:
Accuracy: 0.91441833478387
sensitivity: 0.951797385620915
specificity: 0.8938371569950517
precision: 0.8315488936473947
Confusion matrix:
[[1987  236]
 [  59 1165]]
------------------------
val inference
------------------------
val cell scores:
Accuracy: 0.6211111111111111
sensitivity: 0.5382513661202186
specificity: 0.6779026217228464
precision: 0.5338753387533876
Confusion matrix:
[[362 172]
 [169 197]]
------------------------
Test inference
------------------------
Test cell scores:
Accuracy: 0.5105766444508838
sensitivity: 0.47468354430379744
specificity: 0.5746569814366425
precision: 0.6658211794546608
Confusion matrix:
[[ 712  527]
 [1162 1050]]
------------------------
